In [1]:
Key = 'F240513147'

In [2]:
import requests
import xmltodict
import tkinter as tk
from tkinter import ttk
import pandas as pd
from pandas import json_normalize
import xml.etree.ElementTree as ET

In [3]:
# prodcd : 제품구분(휘발유:B027, 경유:D047, 고급휘발유: B034, 실내등유:C004, 자동차부탄: K015)
# area : 지역구분 (미입력시 전국, 시도코드(2자리):해당시도 기준, 시군코드(4자리):해당시군 기준)
# 지역 코드 : http://www.opinet.co.kr/api/areaCode.do?out=xml&code=XXXXXX&area=01

## 지역코드

In [4]:
url = f'http://www.opinet.co.kr/api/areaCode.do?out=xml&code={Key}'
area_result = xmltodict.parse(requests.get(url).content)
area_result

{'RESULT': {'OIL': [{'AREA_CD': '01', 'AREA_NM': '서울'},
   {'AREA_CD': '02', 'AREA_NM': '경기'},
   {'AREA_CD': '03', 'AREA_NM': '강원'},
   {'AREA_CD': '04', 'AREA_NM': '충북'},
   {'AREA_CD': '05', 'AREA_NM': '충남'},
   {'AREA_CD': '06', 'AREA_NM': '전북'},
   {'AREA_CD': '07', 'AREA_NM': '전남'},
   {'AREA_CD': '08', 'AREA_NM': '경북'},
   {'AREA_CD': '09', 'AREA_NM': '경남'},
   {'AREA_CD': '10', 'AREA_NM': '부산'},
   {'AREA_CD': '11', 'AREA_NM': '제주'},
   {'AREA_CD': '14', 'AREA_NM': '대구'},
   {'AREA_CD': '15', 'AREA_NM': '인천'},
   {'AREA_CD': '16', 'AREA_NM': '광주'},
   {'AREA_CD': '17', 'AREA_NM': '대전'},
   {'AREA_CD': '18', 'AREA_NM': '울산'},
   {'AREA_CD': '19', 'AREA_NM': '세종'}]}}

In [7]:
area_df = json_normalize(area_result['RESULT']['OIL'])
area_df

,AREA_CD,AREA_NM
0,01,서울
1,02,경기
2,03,강원
3,04,충북
4,05,충남
5,06,전북
6,07,전남
7,08,경북
8,09,경남
9,10,부산


## 지역별 최저가 20개

In [6]:
all_results = []

# 각 지역에 대해 데이터를 가져오고, AREA_NM을 함께 결과에 추가하여 리스트에 저장
for area_data in area_result['RESULT']['OIL']:
    area_cd = area_data['AREA_CD']
    area_nm = area_data['AREA_NM']
    url = f'http://www.opinet.co.kr/api/lowTop10.do?out=xml&code={Key}&prodcd=B027&area={area_cd}&cnt=20'
    response = xmltodict.parse(requests.get(url).content)
    result = json_normalize(response['RESULT']['OIL'])
    result['AREA_NM'] = area_nm
    
    all_results.append(result)

# 결과 출력
final_result = pd.concat(all_results, ignore_index=True)
final_result

,UNI_ID,PRICE,POLL_DIV_CO,OS_NM,VAN_ADR,NEW_ADR,GIS_X_COOR,GIS_Y_COOR,AREA_NM
0,A0000686,1624,HDO,(주)디오티디 은평유니콘주유소,서울 은평구 진관동 73-22,서울 은평구 통일로 1151 (진관동),304199.00000,560741.00000,서울
1,A0000519,1644,RTO,화곡역주유소,서울 강서구 화곡동 1076-14,서울 강서구 강서로 154 (화곡동),297830.02778,549333.51356,서울
2,A0000549,1644,HDO,이케이에너지(주) 강서주유소,서울 강서구 화곡동 983-1,서울 강서구 화곡로 273 (화곡동),298318.03900,550276.26480,서울
3,A0000827,1645,RTO,신성주유소,서울 구로구 구로본동461-13,서울 구로구 구로동로 187 (구로제2동),301368.50000,544253.80000,서울
4,A0000856,1645,SOL,씨앤에스유통(주) 구로드림주유소,서울 구로구 온수동 35-3,서울 구로구 경인로 41 (온수동),296439.00000,543853.00000,서울
...,...,...,...,...,...,...,...,...,...
335,A0033031,1665,RTO,대영에너지㈜세종지점 다온알뜰주유소,세종특별자치시 아름동 816,세종특별자치시 세종로 1328 (아름동),332534.88377,435810.66347,세종
336,A0033332,1665,RTO,아성알뜰주유소,세종특별자치시 다정동 399,세종특별자치시 다정북로 40,331676.30143,433241.21627,세종
337,A0013941,1665,SOL,홍대셀프주유소,세종특별자치시 조치원읍 신안리 78-2,세종특별자치시 조치원읍 세종로 2654,336760.71853,447103.16147,세종
338,A0032970,1665,SKE,고운self주유소,세종특별자치시 고운동 1404,세종특별자치시 마음안로 136 (고운동),331531.07989,435092.40031,세종


In [66]:
final_result.to_csv('LowestPrice.csv', index=False, encoding="utf-8-sig")

# 아래는 필요없음

In [41]:
df = json_normalize(result['RESULT']['OIL'])
df

,UNI_ID,PRICE,POLL_DIV_CO,OS_NM,VAN_ADR,NEW_ADR,GIS_X_COOR,GIS_Y_COOR
0,A0014389,1649,SKE,행복주유소,세종시 부강면 등곡리 493,세종시 부강면 시목부강로 423,345147.46678,433879.18432
1,A0014400,1654,RTO,신안주유소,세종특별자치시 조치원읍 신안리 134-5,세종특별자치시 조치원읍 세종로 2678,336732.10000,447361.50000
2,A0015022,1654,ETC,굿모닝주유소,세종특별자치시 조치원읍 신안리 207-6,세종특별자치시 조치원읍 세종로 2712,336663.40000,447677.60000
3,A0014276,1655,GSC,흥진셀프주유소,세종시 조치원읍 신안리 35-26,세종시 조치원읍 세종로 2563,336909.00000,446190.00000
4,A0014198,1655,HDO,그린주유소,세종 조치원읍 신안리 207-10,세종 조치원읍 세종로 2706,336679.77830,447622.58120
5,A0033513,1655,NHO,세종중앙농협주유소,세종특별자치시 연기면 연기리 455,세종특별자치시 연기면 잣띠길 4,335650.97963,438870.55017
6,A0014238,1655,SKE,신원self주유소,세종특별자치시 조치원읍 신안리 88-1,세종특별자치시 조치원읍 세종로 2562,336960.13490,446218.15820
7,A0014189,1655,RTO,충청에너지주유소,세종특별자치시 전의면 유천리 547-1,세종특별자치시 전의면 세종로 3894,327482.36290,453449.79090
8,A0016460,1655,SKE,봉암주유소,세종시 연서면 봉암리 620-93,세종시 연서면 세종로 1899,335870.00000,440054.00000
9,A0015490,1655,SOL,베스트주유소,세종 조치원읍 신안리 210,세종 조치원읍 세종로 2749,336518.60000,448008.50000


## 최저가 주유소

In [28]:
url = f'http://www.opinet.co.kr/api/lowTop10.do?out=xml&code={Key}&prodcd=B027&area=0101&cnt=20'
result = xmltodict.parse(requests.get(url).content)
result

{'RESULT': {'OIL': [{'UNI_ID': 'A0009433',
    'PRICE': '1685',
    'POLL_DIV_CO': 'SKE',
    'OS_NM': '(주)중앙에너비스 혜화주유소',
    'VAN_ADR': '서울 종로구 혜화동 102-2',
    'NEW_ADR': '서울 종로구 창경궁로35길 1',
    'GIS_X_COOR': '311970.75220',
    'GIS_Y_COOR': '554223.60260'},
   {'UNI_ID': 'A0000063',
    'PRICE': '1724',
    'POLL_DIV_CO': 'HDO',
    'OS_NM': 'HD현대오일뱅크㈜직영 재동주유소',
    'VAN_ADR': '서울 종로구 경운동 96-18',
    'NEW_ADR': '서울 종로구 율곡로 58 (경운동)',
    'GIS_X_COOR': '310612.51520',
    'GIS_Y_COOR': '553171.45900'},
   {'UNI_ID': 'A0000064',
    'PRICE': '1869',
    'POLL_DIV_CO': 'SOL',
    'OS_NM': '안풍주유소',
    'VAN_ADR': '서울 종로구 홍지동 126-1',
    'NEW_ADR': '서울 종로구 자하문로 303',
    'GIS_X_COOR': '308281.20000',
    'GIS_Y_COOR': '555728.70000'},
   {'UNI_ID': 'A0000035',
    'PRICE': '1869',
    'POLL_DIV_CO': 'HDO',
    'OS_NM': '㈜지에스이앤알 평창주유소',
    'VAN_ADR': '서울 종로구 평창동 80-1',
    'NEW_ADR': '서울 종로구 평창문화로 135 (평창동)',
    'GIS_X_COOR': '309663.14286',
    'GIS_Y_COOR': '556869.15721'},
   {'UNI_I

In [24]:
df = json_normalize(result['RESULT']['OIL'])

df = df[['OS_NM', 'PRICE', 'VAN_ADR', 'NEW_ADR']]


In [25]:
df

,OS_NM,PRICE,VAN_ADR,NEW_ADR
0,케이티엑스(주)명성셀프주유소,1575,울산 울주군 웅촌면 대대리145-9,울산 울주군 웅촌면 웅촌로 217
1,율리셀프주유소,1585,울산 울주군 청량면 율리 772-3,울산 울주군 청량면 웅촌로 1044
2,기장대로 셀프주유소 와이케이에너지㈜,1585,부산 기장군 기장읍 청강리 539,부산 기장군 기장읍 기장대로 343
3,삼양주유소,1585,부산 해운대구 송정동 80-4,부산 해운대구 해운대로 1249 (송정동)
4,태이주유소,1585,울산 울주군 웅촌면 대대리 1084-18,울산 울주군 웅촌면 웅촌로 81
5,웅촌구도일주유소,1585,울산 울주군 웅촌면 곡천리 1-19,울산 울주군 웅촌면 웅촌로 504
6,(주)세일 송정역주유소,1585,부산 해운대구 송정동 109-3,부산 해운대구 해운대로 1217
7,율선주유소,1585,울산 울주군 청량면 율리 767-2,울산 울주군 청량면 웅촌로 1035
8,㈜영원석유 하늘주유소,1585,부산 해운대구 송정동 105-8,부산 해운대구 해운대로 1184
9,오시리아주유소,1585,부산 해운대구 송정동 86-7,부산 해운대구 해운대로 1226 (송정동)


In [ ]:
https://www.opinet.co.kr/api/avgLastWeek.do?prodcd=B027&code={Key}&out=xml

In [9]:
url = f'https://www.opinet.co.kr/api/avgLastWeek.do?code={Key}&out=xml'
week_result = xmltodict.parse(requests.get(url).content)
week_result

{'RESULT': {'OIL': [{'WEEK': '2024052',
    'STA_DT': '20240505',
    'END_DT': '20240511',
    'AREA_CD': '00',
    'PRODCD': 'B027',
    'PRICE': '1711.72'},
   {'WEEK': '2024052',
    'STA_DT': '20240505',
    'END_DT': '20240511',
    'AREA_CD': '00',
    'PRODCD': 'B034',
    'PRICE': '1951.76'},
   {'WEEK': '2024052',
    'STA_DT': '20240505',
    'END_DT': '20240511',
    'AREA_CD': '00',
    'PRODCD': 'C004',
    'PRICE': '1367.35'},
   {'WEEK': '2024052',
    'STA_DT': '20240505',
    'END_DT': '20240511',
    'AREA_CD': '00',
    'PRODCD': 'D047',
    'PRICE': '1560.77'},
   {'WEEK': '2024052',
    'STA_DT': '20240505',
    'END_DT': '20240511',
    'AREA_CD': '00',
    'PRODCD': 'K015',
    'PRICE': '970.21'}]}}